In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install pyspark and findspark

In [ ]:
!pip install pyspark


!pip install findspark

# Declare library

In [ ]:
import findspark
findspark.init


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.functions import explode
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from IPython.display import display
from IPython.display import FileLink, FileLinks
import os
import pandas as pd
from pandas import DataFrame
pd.options.display.max_columns = None
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from pyspark.sql.types import ArrayType, IntegerType, DoubleType
from pyspark.sql.window import Window

from datetime import date
from dateutil.relativedelta import relativedelta

#from mmlspark.lightgbm import LightGBMClassifier ,  LightGBMClassificationModel
#from mmlspark.vw import VowpalWabbitFeaturizer, VowpalWabbitClassifier

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Summarizer #used to summarize vectors
from pyspark.ml.feature import VectorAssembler , VectorSlicer , HashingTF, IDF, Tokenizer , StopWordsRemover,CountVectorizer
from pyspark.ml.linalg import Vectors , DenseVector
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier , LogisticRegression


spark = SparkSession.builder.appName("Pyspark_Machine_Learning").getOrCreate()

sc= spark.sparkContext
sqlContext= SQLContext(sc)


# Read data in csv format and show some of its info

In [ ]:
df=spark.read\
.option("header", "true")\
.option("inferSchema", "true")\
.csv('../input/water-potability/water_potability.csv')

df.show(10)
df.count()
df.dtypes



In [ ]:
#df=df.dropna()
#df.count()
df.select('Potability').describe().show()

# Impute the dataset with mean (to prevent missing values）

In [ ]:
from pyspark.ml.feature import Imputer
imputeCols = ["ph", "Hardness", "Solids", "Chloramines", "Sulfate","Conductivity","Organic_carbon","Trihalomethanes","Turbidity"]
imputer = Imputer(strategy="mean", inputCols=imputeCols, outputCols=imputeCols)
imputerModel = imputer.fit(df)
imputedDF = imputerModel.transform(df)

imputedDF.show(10)

# Put all feature columns into vector

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureCols = ["ph", "Hardness", "Solids", "Chloramines", "Sulfate","Conductivity","Organic_carbon","Trihalomethanes","Turbidity"]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
df_trans = assembler.transform(imputedDF)

df_trans.select('features','Potability').show(10)

In [ ]:
df_trans.select('features','Potability').filter(df_trans.Potability==1).show(10)

# Import logistic regression of pyspark ml, since this is a classification problem

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Potability", featuresCol="features")
lrModel = lr.fit(df_trans)

df_predict = lrModel.transform(df_trans)
df_predict.show(5)

# Check which metrics the logistic model have, then choose one to have a look

In [ ]:
[attr for attr in dir(lrModel.summary) if attr[0] != "_"]

In [ ]:
lrModel.summary.truePositiveRateByLabel

# Evaluate the model (roc and pr curve)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='Potability')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(df_predict, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(df_predict, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))